In [1]:
import pyspark
sc = pyspark.SparkContext('local[2]')
import pandas as pd, numpy as np

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tip").getOrCreate()

reviews_df =spark.read.json('/Users/vijethapv/Downloads/Documents/INF553_DataMining/yelp_dataset/yelp_academic_dataset_review.json')
business_df =spark.read.json('/Users/vijethapv/Downloads/Documents/INF553_DataMining/yelp_dataset/yelp_academic_dataset_business.json')
users_df =spark.read.json('/Users/vijethapv/Downloads/Documents/INF553_DataMining/yelp_dataset/yelp_academic_dataset_user.json')



In [3]:
users_friends_list = users_df['user_id','friends']

In [4]:
# for row in users_friends_list.loc[users_friends_list.friends.isnull(), 'friends'].index:
#     users_friends_list.at[row, 'friends'] = []
from pyspark.sql.functions import col
# users_with_friends = users_friends_list.where(col("friends") is not 'None')
users_with_friends = users_friends_list.filter(users_friends_list.friends!='None')
# users_with_friends =users_friends_list.na.drop(subset=["friends"])

In [5]:
users_with_friends.count()

879891

In [7]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
# users_friends_list.fillna({'friends':[]})
slen = udf(lambda s: len(s.split(",")), IntegerType())

users_friends_list_2 = users_with_friends.withColumn("friends_cnt", slen(users_with_friends.friends))


In [8]:
users_friends_list_2.count()

879891

In [ ]:
businesses_in_arizona = business_df.where(business_df.state=='AZ')


businesses_in_arizona=businesses_in_arizona.withColumnRenamed("name", "business_name").withColumnRenamed("stars", "business_stars").withColumnRenamed("review_count", "business_review_count")

reviews_df = reviews_df.withColumnRenamed("stars", "reviews_star").withColumnRenamed("cool", "reviews_cool").withColumnRenamed("funny", "reviews_funny").withColumnRenamed("useful", "reviews_useful")

# df1.join(df2, df1("col1") === df2("col1"), "left_outer")
business_to_review_df= businesses_in_arizona.join(reviews_df,businesses_in_arizona.business_id==reviews_df.business_id,"left_outer").drop(reviews_df.business_id)

# Dont execute below cells

Considering the users who have reviewed businesses in Arizona , we may further narrow this down to users who have provided atleast 10 or > reviews for businesses in Arizona , to say that they've stayed there.

users_from_arizona = business_to_review_df.select("user_id").distinct() # assumption that these users are from Arizona

users_from_arizona.count()

In [23]:
users_from_arizona_more_details_with_friends.count()

262619

In [29]:
users_from_arizona_more_details_with_friends.columns

['user_id',
 'average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'elite',
 'fans',
 'friends',
 'funny',
 'name',
 'review_count',
 'useful',
 'yelping_since']

In [31]:
users_in_arizona_friends_list = users_from_arizona_more_details_with_friends['user_id','friends']

In [79]:
users_from_arizona_pd =users_from_arizona_more_details_with_friends.limit(200000).toPandas()

In [42]:
import pandas as pd

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [80]:
limited_users = users_from_arizona_more_details_with_friends.limit(200000) # list of users and cross check results
limited_users.count()

200000

In [93]:
consolidated_data= limited_users.join(business_to_review_df, ['user_id'])

In [61]:
consolidated_data.count()

1030289

In [91]:
business_to_review_df.columns

['address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'business_name',
 'neighborhood',
 'postal_code',
 'business_review_count',
 'business_stars',
 'state',
 'business_id',
 'reviews_cool',
 'date',
 'reviews_funny',
 'review_id',
 'reviews_star',
 'text',
 'reviews_useful',
 'user_id']

In [94]:
consolidated_data.columns

['user_id',
 'average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'elite',
 'fans',
 'friends',
 'funny',
 'name',
 'review_count',
 'useful',
 'yelping_since',
 'address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'business_name',
 'neighborhood',
 'postal_code',
 'business_review_count',
 'business_stars',
 'state',
 'reviews_cool',
 'date',
 'reviews_funny',
 'review_id',
 'reviews_star',
 'text',
 'reviews_useful']

In [100]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def array_to_string(my_list):
    return '[' + ','.join([str(elem) for elem in my_list]) + ']'

array_to_string_udf = udf(array_to_string,StringType())

# df = df.withColumn('column_as_str',array_to_string_udf(d["column_as_array"]))

In [108]:
consolidated_data=consolidated_data.withColumn("attributes_as_str", array_to_string_udf(consolidated_data["attributes"]))
consolidated_data=consolidated_data.withColumn("hours_as_str", array_to_string_udf(consolidated_data["hours"]))

consolidated_data.drop("attributes").drop("hours").repartition(1).write.format('com.databricks.spark.csv').save('mycsv.csv')

AnalysisException: 'path file:/Users/vijethapv/mycsv.csv already exists.;'

In [110]:
# consolidated_data.drop("attributes").drop("hours").to_csv("arizona.csv")

AttributeError: 'DataFrame' object has no attribute 'to_csv'

In [107]:
#consolidated_data.write.csv('res.csv')

Py4JJavaError: An error occurred while calling o848.csv.
: java.lang.UnsupportedOperationException: CSV data source does not support struct<AcceptsInsurance:string,AgesAllowed:string,Alcohol:string,Ambience:string,BYOB:string,BYOBCorkage:string,BestNights:string,BikeParking:string,BusinessAcceptsBitcoin:string,BusinessAcceptsCreditCards:string,BusinessParking:string,ByAppointmentOnly:string,Caters:string,CoatCheck:string,Corkage:string,DietaryRestrictions:string,DogsAllowed:string,DriveThru:string,GoodForDancing:string,GoodForKids:string,GoodForMeal:string,HairSpecializesIn:string,HappyHour:string,HasTV:string,... 15 more fields> data type.
	at org.apache.spark.sql.execution.datasources.csv.CSVUtils$.org$apache$spark$sql$execution$datasources$csv$CSVUtils$$verifyType$1(CSVUtils.scala:127)
	at org.apache.spark.sql.execution.datasources.csv.CSVUtils$$anonfun$verifySchema$1.apply(CSVUtils.scala:131)
	at org.apache.spark.sql.execution.datasources.csv.CSVUtils$$anonfun$verifySchema$1.apply(

In [121]:
df_bus_user_review=consolidated_data.drop("attributes").drop("hours").drop("friends").drop("state").drop("text")

In [124]:
df_bus_user_review.toDF.

DataFrame[user_id: string, average_stars: double, compliment_cool: bigint, compliment_cute: bigint, compliment_funny: bigint, compliment_hot: bigint, compliment_list: bigint, compliment_more: bigint, compliment_note: bigint, compliment_photos: bigint, compliment_plain: bigint, compliment_profile: bigint, compliment_writer: bigint, cool: bigint, elite: string, fans: bigint, funny: bigint, name: string, review_count: bigint, useful: bigint, yelping_since: string, address: string, business_id: string, categories: string, city: string, is_open: bigint, latitude: double, longitude: double, business_name: string, neighborhood: string, postal_code: string, business_review_count: bigint, business_stars: double, reviews_cool: bigint, date: string, reviews_funny: bigint, review_id: string, reviews_star: bigint, reviews_useful: bigint, attributes_as_str: string, hours_as_str: string]

In [131]:
experimental_users = df_bus_user_review.limit(100)

In [132]:
df=experimental_users.toPandas()

Py4JJavaError: An error occurred while calling o1278.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 235.0 failed 1 times, most recent failure: Lost task 0.0 in stage 235.0 (TID 9569, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/vijethapv/spark-2.3.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/Users/vijethapv/spark-2.3.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/vijethapv/spark-2.3.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 324, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/Users/vijethapv/spark-2.3.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 139, in dump_stream
    for obj in iterator:
  File "/

In [ ]:
#df.write.format('com.databricks.spark.csv').save('mycsv.csv')

In [136]:
limited_users.filter(limited_users.review_count >10).count()

77414

In [137]:
users_from_arizona_more_details_with_friends.filter(limited_users.review_count >10).count()

101918

In [138]:
from pyspark.sql.functions import mean, min, max
users_from_arizona_more_details_with_friends.select([mean('review_count'), min('review_count'), max('review_count')]).show()

+-----------------+-----------------+-----------------+
|avg(review_count)|min(review_count)|max(review_count)|
+-----------------+-----------------+-----------------+
|32.49878721646187|                0|            11942|
+-----------------+-----------------+-----------------+



In [144]:
users_from_arizona_more_details_with_friends.filter(limited_users.review_count >35).count()

42551

In [145]:
limited_users_avg_32= users_from_arizona_more_details_with_friends.filter(limited_users.review_count >35)

In [148]:
#limited_users_avg_32.toPandas().to_csv('limited_users_avg_35reviews.csv')

In [149]:
consolidated_data= limited_users_avg_32.join(business_to_review_df, ['user_id'])

In [152]:
consolidated_data=consolidated_data.drop('attributes').drop('address').drop('hours').drop('is_open').drop('neighborhood').drop('text').drop('state').drop('friends')

In [153]:
#consolidated_data.toPandas().to_csv('all_details_users.csv')

In [156]:
consolidated_data.select('business_id').distinct().count()

44440

In [157]:
consolidated_data.select('categories').distinct().count()

24507

In [158]:
business_df.select('categories').distinct().count()

85057

In [163]:
consolidated_data.groupBy('categories').count().show()

+--------------------+-----+
|          categories|count|
+--------------------+-----+
|American (Traditi...|  459|
|Bars, Sports Bars...|  133|
|Arts & Crafts, Sh...|   12|
|Drugstores, Depar...|   40|
|Arts & Entertainm...|  614|
|Halal, Indian, Bu...|  103|
|Restaurants, Pizz...|  100|
|Barbers, Waxing, ...|   30|
|Soup, Sandwiches,...|   14|
|Seafood, American...|   74|
|Specialty Food, W...|    1|
|Hair Removal, Las...|   36|
|Home Decor, Child...|   14|
|Arts & Crafts, Ap...|   19|
|American (New), S...|   75|
|Fashion, Men's Cl...|   12|
|Food, Event Plann...|   45|
|Automotive, Truck...|    3|
|Mexican, Bars, Da...|  145|
|Restaurants, Chic...|   25|
+--------------------+-----+
only showing top 20 rows



In [166]:
consolidated_data.filter(consolidated_data.review_count==0).count()

0

In [167]:
consolidated_data.groupBy('user_id').count().show()

+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|-3i9bhfvrM3F1wsC9...|    3|
|-55DgUo52I3zW9Rxk...|   26|
|-9da1xk7zgnnfO1uT...|    1|
|-GKEFg_92pp0q842c...|   12|
|01HpNrCSLNWbQG9Fv...|    1|
|0b3FXt-EgiwMvS4sc...|    1|
|0dmMgfbP23TzB6jNq...|    2|
|0lbUv-31EVxr8isqk...|    5|
|0qQXako5doigx3X7J...|    1|
|0y8ORuC2X1i1UF6SG...|    3|
|1p4wP0QzgdWXWYnRS...|    4|
|1uiAOPCQf54rAwQWi...|    4|
|2OtKt_7AZkfPXioU3...|   31|
|2YshwnCqBvT4i2zhU...|    3|
|2boQDeHxopolPtJhV...|    1|
|2oD-vur-aVCsOVdW0...|    2|
|37FUZQAVQNT99CjxX...|    1|
|3pMczoCBOSKBcqMhV...|   19|
|3rWUVww0VKB8_LC77...|    3|
|46wwNRiBGIAJuPlk8...|   13|
+--------------------+-----+
only showing top 20 rows



In [168]:
user_review_count = consolidated_data.groupBy('user_id').count()

In [169]:
user_review_count.columns

['user_id', 'count']

In [170]:
user_review_count.select([mean('count'), min('count'), max('count')]).show()

+-----------------+----------+----------+
|       avg(count)|min(count)|max(count)|
+-----------------+----------+----------+
|14.48976522290898|         1|      1409|
+-----------------+----------+----------+



In [175]:
user_review_count.filter(user_review_count.count() > 14).count()

TypeError: condition should be string or Column

In [218]:
cons_all_data =users_from_arizona_more_details_with_friends.join(business_to_review_df, ['user_id'])

In [177]:
group_by_user =cons_all_data.groupBy('user_id').count()

In [219]:
group_by_user.select([mean('count'), min('count'), max('count')]).show()

Py4JJavaError: An error occurred while calling o2197.showString.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange SinglePartition
+- *(10) HashAggregate(keys=[], functions=[partial_avg(count#5500L), partial_min(count#5500L), partial_max(count#5500L)], output=[sum#6604, count#6605L, min#6606L, max#6607L])
   +- *(10) HashAggregate(keys=[user_id#124], functions=[count(1)], output=[count#5500L])
      +- *(10) HashAggregate(keys=[user_id#124], functions=[partial_count(1)], output=[user_id#124, count#5531L])
         +- *(10) Project [user_id#124]
            +- *(10) SortMergeJoin [user_id#124], [user_id#5410], Inner
               :- *(6) Project [user_id#124]
               :  +- *(6) SortMergeJoin [user_id#124], [user_id#196], Inner
               :     :- *(3) Sort [user_id#124 ASC NULLS FIRST], false, 0
               :     :  +- *(3) HashAggregate(keys=[user_id#124], functions=[], output=[user_id#124])
               :     :     +- Exchange h

In [179]:
users_from_arizona_more_details_with_friends.count()

262619

In [ ]:
cons_all_data.count()

In [182]:
cons_all_data.groupBy('user_id').count().show()

+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|-3i9bhfvrM3F1wsC9...|    3|
|-55DgUo52I3zW9Rxk...|   26|
|-7fIoUgPboApCfkG0...|    2|
|-9da1xk7zgnnfO1uT...|    1|
|-D8Zi00T-ivGzQNc-...|    1|
|-Ef8EdRF7yDxrCXIE...|    1|
|-GKEFg_92pp0q842c...|   12|
|-JJo0A9pk6-l6nryr...|    2|
|-KPZpkZEuKaMYQXHW...|    1|
|-d5jZCMlYOEa-jQDU...|    1|
|-dV1Z7attfgdpyNpZ...|    5|
|-dgy_amGIhXwVliYL...|    1|
|-imIGQsuvUSWVm1d4...|    1|
|-inp099-1gsJF3KPa...|   18|
|-rhuuAsGt9B0J6xW3...|    1|
|-t7ZPfmhcafvYsPK2...|    2|
|01HpNrCSLNWbQG9Fv...|    1|
|03Q0N0Sea4wQVbIPD...|    1|
|05I3tm2UdUAqhiVLz...|    1|
|076xpLh68vAcpUHSw...|    2|
+--------------------+-----+
only showing top 20 rows



In [220]:
group_by_user_count =cons_all_data.groupBy('user_id').agg({"*": "count"})

In [221]:
group_by_user_count.columns

['user_id', 'count(1)']

In [222]:
group_by_user_count.select([mean('count(1)'), min('count(1)'), max('count(1)')]).show()

Py4JJavaError: An error occurred while calling o2254.showString.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange SinglePartition
+- *(10) HashAggregate(keys=[], functions=[partial_avg(count(1)#6654L), partial_min(count(1)#6654L), partial_max(count(1)#6654L)], output=[sum#6680, count#6681L, min#6682L, max#6683L])
   +- *(10) HashAggregate(keys=[user_id#124], functions=[count(1)], output=[count(1)#6654L])
      +- *(10) HashAggregate(keys=[user_id#124], functions=[partial_count(1)], output=[user_id#124, count#6685L])
         +- *(10) Project [user_id#124]
            +- *(10) SortMergeJoin [user_id#124], [user_id#6536], Inner
               :- *(6) Project [user_id#124]
               :  +- *(6) SortMergeJoin [user_id#124], [user_id#196], Inner
               :     :- *(3) Sort [user_id#124 ASC NULLS FIRST], false, 0
               :     :  +- *(3) HashAggregate(keys=[user_id#124], functions=[], output=[user_id#124])
               :     :     +

In [216]:
group_by_user_count.approxQuantile("count(1)",[0.5],0.25)

Py4JJavaError: An error occurred while calling o2106.approxQuantile.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(user_id#124, 200)
+- *(2) HashAggregate(keys=[user_id#124], functions=[], output=[user_id#124])
   +- *(2) Project [user_id#124]
      +- *(2) BroadcastHashJoin [business_id#142], [business_id#116], Inner, BuildLeft
         :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true]))
         :  +- *(1) Project [business_id#142]
         :     +- *(1) Filter (isnotnull(state#154) && (state#154 = AZ))
         :        +- *(1) FileScan json [business_id#142,state#154] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/Users/vijethapv/Downloads/Documents/INF553_DataMining/yelp_dataset/yelp_a..., PartitionFilters: [], PushedFilters: [IsNotNull(state), EqualTo(state,AZ)], ReadSchema: struct<business_id:string,state:string>
         +- *(2) Project [business_id#116, user_id#124]
       

In [190]:
grouped =group_by_user_count.select([mean('count(1)'), min('count(1)'), max('count(1)')])

In [194]:
group_by_user_count = group_by_user_count.withColumnRenamed("count(1)", "business_review_count_in_arizona")

In [195]:
group_by_user_count.columns

['user_id', 'business_review_count_in_arizona']

In [197]:
group_by_user_count.filter(group_by_user_count.business_review_count_in_arizona > 5).count()

49589

In [206]:
filtered_group= group_by_user_count.filter(group_by_user_count.business_review_count_in_arizona >5)

In [214]:
filtered_group.count()

Py4JJavaError: An error occurred while calling o1781.count.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange SinglePartition
+- *(10) HashAggregate(keys=[], functions=[partial_count(1)], output=[count#6464L])
   +- *(10) Project
      +- *(10) Filter (count(1)#5948L > 5)
         +- *(10) HashAggregate(keys=[user_id#124], functions=[count(1)], output=[count(1)#5948L])
            +- *(10) HashAggregate(keys=[user_id#124], functions=[partial_count(1)], output=[user_id#124, count#5985L])
               +- *(10) Project [user_id#124]
                  +- *(10) SortMergeJoin [user_id#124], [user_id#5410], Inner
                     :- *(6) Project [user_id#124]
                     :  +- *(6) SortMergeJoin [user_id#124], [user_id#196], Inner
                     :     :- *(3) Sort [user_id#124 ASC NULLS FIRST], false, 0
                     :     :  +- *(3) HashAggregate(keys=[user_id#124], functions=[], output=[user_id#124])
                     : 

In [207]:
cons_all_data = filtered_group.join(business_to_review_df, ['user_id'])

In [208]:
cons_all_data_rmved = cons_all_data.drop('attributes').drop('address').drop('hours').drop('is_open').drop('neighborhood').drop('text').drop('state').drop('friends')

In [209]:
cons_all_data_rmved.toPandas().to_csv('arizona_all_details_users.csv')

Py4JJavaError: An error occurred while calling o1792.collectToPython.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(user_id#124, 200)
+- *(2) HashAggregate(keys=[user_id#124], functions=[], output=[user_id#124])
   +- *(2) Project [user_id#124]
      +- *(2) BroadcastHashJoin [business_id#142], [business_id#116], Inner, BuildLeft
         :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true]))
         :  +- *(1) Project [business_id#142]
         :     +- *(1) Filter (isnotnull(state#154) && (state#154 = AZ))
         :        +- *(1) FileScan json [business_id#142,state#154] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/Users/vijethapv/Downloads/Documents/INF553_DataMining/yelp_dataset/yelp_a..., PartitionFilters: [], PushedFilters: [IsNotNull(state), EqualTo(state,AZ)], ReadSchema: struct<business_id:string,state:string>
         +- *(2) Project [business_id#116, user_id#124]
      

# Execute again from below

In [16]:
from pyspark.sql.functions import mean, min, max
grouped_users = business_to_review_df.groupby("user_id").agg({'*':'count'})


In [17]:
business_to_review_df.columns

['address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'business_name',
 'neighborhood',
 'postal_code',
 'business_review_count',
 'business_stars',
 'state',
 'reviews_cool',
 'date',
 'reviews_funny',
 'review_id',
 'reviews_star',
 'text',
 'reviews_useful',
 'user_id',
 'text_count']

In [18]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import col, split

size_ = udf(lambda xs: len(xs), IntegerType())
business_to_review_df = business_to_review_df.withColumn("text_count", size_(col("text")))



In [19]:
users_more_details = grouped_users.join(users_df, ['user_id'])
users_who_have_friends = users_more_details.filter(users_more_details.friends!='None')


In [20]:
users_who_have_friends.approxQuantile("count(1)",[0.5],0.25)
users_who_have_friends=users_who_have_friends.withColumnRenamed("count(1)","business_reviewed_count")
users_who_have_friends.select([mean('business_reviewed_count'), min('business_reviewed_count'), max('business_reviewed_count')]).show()
limited_users =users_who_have_friends.filter(users_who_have_friends.business_reviewed_count > 5 )
users_all_Details_bR = limited_users.join(business_to_review_df, ['user_id'])
users_details_all_bus_open_Review =users_all_Details_bR.filter(users_all_Details_bR.is_open==1)


+----------------------------+----------------------------+----------------------------+
|avg(business_reviewed_count)|min(business_reviewed_count)|max(business_reviewed_count)|
+----------------------------+----------------------------+----------------------------+
|           5.128551247244107|                           1|                        1409|
+----------------------------+----------------------------+----------------------------+



In [21]:
users_details_all_bus_open_Review.columns

['user_id',
 'business_reviewed_count',
 'average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'elite',
 'fans',
 'friends',
 'funny',
 'name',
 'review_count',
 'useful',
 'yelping_since',
 'address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'business_name',
 'neighborhood',
 'postal_code',
 'business_review_count',
 'business_stars',
 'state',
 'reviews_cool',
 'date',
 'reviews_funny',
 'review_id',
 'reviews_star',
 'text',
 'reviews_useful',
 'text_count']

In [22]:
users_details_all_bus_open_Review = users_details_all_bus_open_Review.withColumn("yelping_since_years", (2018-split(col('yelping_since'),"-")[0]))

In [30]:
import pyspark.sql.functions as funcs

In [32]:
users_details_all_bus_open_Review = users_details_all_bus_open_Review.withColumn("friends_count",funcs.size(split(col("friends"), ",\s*"))).withColumn("elite_count", funcs.when(col("elite") == 'None',0).otherwise(funcs.size(split(col("elite"), ",\s*"))))

# users_details_all_bus_open_Review = users_details_all_bus_open_Review.

# Old File

In [ ]:
users_details_all_bus_open_Review=users_details_all_bus_open_Review.drop('attributes').drop('address').drop('hours').drop('is_open').drop('neighborhood').drop('state').drop('friends').drop('text')
users_details_all_bus_open_Review.toPandas().to_csv('arizona_all_details_users_49k_open_businesses.csv')

# New File

In [42]:
users_details_all_bus_open_Review=users_details_all_bus_open_Review.drop('city').drop('reviews_cool').drop('reviews_funny').drop('business_name').drop('postal_code').drop('name').drop('elite').drop('yelping_since').drop('attributes').drop('address').drop('hours').drop('is_open').drop('neighborhood').drop('state').drop('friends').drop('text')
users_details_all_bus_open_Review.columns

['user_id',
 'business_reviewed_count',
 'average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'fans',
 'funny',
 'review_count',
 'useful',
 'business_id',
 'categories',
 'latitude',
 'longitude',
 'business_review_count',
 'business_stars',
 'date',
 'review_id',
 'reviews_star',
 'reviews_useful',
 'text_count',
 'yelping_since_years',
 'friends_count',
 'elite_count']

In [44]:
users_details_all_bus_open_Review.toPandas().to_csv('user_business_rating_49k_features_added.csv')

Py4JJavaError: An error occurred while calling o486.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 50.0 failed 1 times, most recent failure: Lost task 0.0 in stage 50.0 (TID 903, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/vijethapv/spark-2.3.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 176, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:298)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:83)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.r